In [1]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px

import boto3
from io import StringIO

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path=r'C:\Users\ambri\Desktop\vscodetraining\Training\FullStack\.env')

from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

# Exploratory Data Analysis to find the top 20 hotels in the top 5 cities

In [2]:
#Read the csv that we create in the scrap notebook
df = pd.read_csv("Kayak_Project_scrap_booking.csv")
df.head()

,city,hotel_lat,hotel_lon,hotel_name,hotel_review,hotel_nbr_review,hotel_address,hotel_facilities,hotel_desc,hotel_url
0,Carcassonne,43.206183545263,2.361151027302,Studio pour 2 avec terrasse au pied de la Cité...,"9,5",174 expériences vécues,"36 Rue Longue, 11000 Carcassonne, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Studio pour 2 avec terrasse au p...,https://www.booking.com/hotel/fr/studio-pour-2...
1,Carcassonne,43.2114588,2.3508581,Marius - studio au cœur de la bastide,"8,0",112 expériences vécues,"9 Ruelle Perrot, 11000 Carcassonne, France","Connexion Wi-Fi gratuite,Chambres non-fumeurs,...",L’hébergement Marius - studio au cœur de la ba...,https://www.booking.com/hotel/fr/marius-studio...
2,Aigues Mortes,43.5672196715711,4.189343601465225,Hôtel Saint Louis,"8,5",1 577 expériences vécues,"10, Rue Amiral Courbet, 30220 Aigues-Mortes, F...","Parking,Spa et centre de bien-être,Connexion W...","Situé dans le cœur historique d'Aigues-Mortes,...",https://www.booking.com/hotel/fr/saint-louis-a...
3,Carcassonne,43.2140544,2.3460563,Chez Pauline & Vincent Côté Canal,"9,8",11 expériences vécues,"19 Avenue Pierre Charles Lespinasse, 11000 Car...","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Chez Pauline & Vincent Côté Cana...,https://www.booking.com/hotel/fr/cote-canal-ca...
4,Carcassonne,43.2082373962379,2.36704409122467,La Rapière,"8,4",3 099 expériences vécues,"9 Montée Gaston Combéléran, 11000 Carcassonne,...","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",La chambre d'hôtes La Rapiere est située à Car...,https://www.booking.com/hotel/fr/la-rapia-re.f...


In [3]:
# Types of columns
df.dtypes

city                object
hotel_lat           object
hotel_lon           object
hotel_name          object
hotel_review        object
hotel_nbr_review    object
hotel_address       object
hotel_facilities    object
hotel_desc          object
hotel_url           object
dtype: object

In [4]:
# Convert float data
df['hotel_lat'] = pd.to_numeric(df['hotel_lat'], errors='coerce')
df['hotel_lon'] = pd.to_numeric(df['hotel_lon'], errors='coerce')

In [5]:
#Clean the str to have only float
df['hotel_nbr_review'] = df['hotel_nbr_review'].str.extract(r'([\d.]+)')
df['hotel_nbr_review'] = pd.to_numeric(df['hotel_nbr_review'])

#Standardization of the object then transform to numeric
df['hotel_review'] = df['hotel_review'].str.replace(',', '.')
df['hotel_review'] = pd.to_numeric(df['hotel_review'], errors='coerce')

# Load the dataframe in a data lake : S3 bucket
session = boto3.Session(aws_access_key_id=os.environ["AWS_ACCESS_KEY"], 
                        aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"])

s3 = session.resource("s3")

bucket = s3.create_bucket(Bucket="citiesinfoskayakjedha")

# Transform the dataframe into a csv. and upload the CSV file to S3
csv = df.to_csv()
put_object = bucket.put_object(Key="all_hotel_info.csv", Body=csv)

df.head()

,city,hotel_lat,hotel_lon,hotel_name,hotel_review,hotel_nbr_review,hotel_address,hotel_facilities,hotel_desc,hotel_url
0,Carcassonne,43.206184,2.361151,Studio pour 2 avec terrasse au pied de la Cité...,9.5,174.0,"36 Rue Longue, 11000 Carcassonne, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Studio pour 2 avec terrasse au p...,https://www.booking.com/hotel/fr/studio-pour-2...
1,Carcassonne,43.211459,2.350858,Marius - studio au cœur de la bastide,8.0,112.0,"9 Ruelle Perrot, 11000 Carcassonne, France","Connexion Wi-Fi gratuite,Chambres non-fumeurs,...",L’hébergement Marius - studio au cœur de la ba...,https://www.booking.com/hotel/fr/marius-studio...
2,Aigues Mortes,43.567220,4.189344,Hôtel Saint Louis,8.5,1.0,"10, Rue Amiral Courbet, 30220 Aigues-Mortes, F...","Parking,Spa et centre de bien-être,Connexion W...","Situé dans le cœur historique d'Aigues-Mortes,...",https://www.booking.com/hotel/fr/saint-louis-a...
3,Carcassonne,43.214054,2.346056,Chez Pauline & Vincent Côté Canal,9.8,11.0,"19 Avenue Pierre Charles Lespinasse, 11000 Car...","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Chez Pauline & Vincent Côté Cana...,https://www.booking.com/hotel/fr/cote-canal-ca...
4,Carcassonne,43.208237,2.367044,La Rapière,8.4,3.0,"9 Montée Gaston Combéléran, 11000 Carcassonne,...","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",La chambre d'hôtes La Rapiere est située à Car...,https://www.booking.com/hotel/fr/la-rapia-re.f...


In [ ]:
# Initialization of the SQL engine
USERNAME = "postgres"
HOSTNAME = os.environ["HOSTNAME"]
PASSWORD = os.environ["PASSWORD"]

engine = create_engine(f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

# Storing the dataframe as SQL tables
df.to_sql('hotel_info', con=engine, if_exists='replace', index=False)

2025-03-02 14:23:49,225 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-02 14:23:49,228 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-02 14:23:49,263 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-02 14:23:49,267 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-02 14:23:49,300 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-02 14:23:49,302 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-02 14:23:49,336 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-02 14:23:49,352 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

198

In [7]:
#Apply a mask with the conditions needed : the review and the nbr of review
conditions = (df['hotel_review']>8.7) & (df['hotel_nbr_review'] > 30)
top_hotels = df[conditions]
top20_hotels = pd.DataFrame(top_hotels.iloc[:20,:]).sort_values(['hotel_review'],ascending=False).reset_index()
top20_hotels

,index,city,hotel_lat,hotel_lon,hotel_name,hotel_review,hotel_nbr_review,hotel_address,hotel_facilities,hotel_desc,hotel_url
0,40,Cassis,43.217600,5.532350,LE SEPT charmant studio aux portes des calanques,9.9,89.0,"7 Avenue des Carriers, 13260 Cassis, France","Piscine extérieure,Parking gratuit,Connexion W...","Situé à Cassis et offrant une vue sur la mer, ...",https://www.booking.com/hotel/fr/le-sept-charm...
1,35,Cassis,43.220452,5.536091,Villa Bobnell,9.6,89.0,"4 Impasse du 8 mai, 13260 Cassis, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’établissement Villa Bobnell se situe à Cassi...,https://www.booking.com/hotel/fr/villa-bobnell...
2,0,Carcassonne,43.206184,2.361151,Studio pour 2 avec terrasse au pied de la Cité...,9.5,174.0,"36 Rue Longue, 11000 Carcassonne, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Studio pour 2 avec terrasse au p...,https://www.booking.com/hotel/fr/studio-pour-2...
3,42,Cassis,43.216099,5.537840,Mistral gagnant - parking privé,9.5,77.0,"25 Rue Frédéric Mistral, 13260 Cassis, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement Mistral gagnant - parking privé ...,https://www.booking.com/hotel/fr/mistral-gagna...
4,51,Saintes Maries de la mer,43.517299,4.374500,"La Villa Vaihéré en Camargue, l'Hibiscus et le...",9.5,110.0,"78, Cabanes de Cambon, 13460 Les Saintes-Marie...","Parking gratuit,Connexion Wi-Fi rapide gratuit...","L’établissement La Villa Vaihéré en Camargue, ...",https://www.booking.com/hotel/fr/la-villa-vaih...
5,21,Saintes Maries de la mer,43.457069,4.430955,"Chambre les flamants, vue sur les étangs, les ...",9.5,62.0,"29 Rue Fanfonne Guillerme, 13460 Les Saintes-M...","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...","L’hébergement Chambre les flamants, vue sur le...",https://www.booking.com/hotel/fr/chambre-les-f...
6,32,Cassis,43.227286,5.533102,"Rêves de vie, studio Port Miou perle rare à Ca...",9.4,116.0,"47 Avenue des Terrasses, 13260 Cassis, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...","Doté d’un bain à remous, l’hébergement Rêves d...",https://www.booking.com/hotel/fr/reves-de-vie....
7,52,Cassis,43.215009,5.530956,La douceur de Cassis,9.4,106.0,"16 Avenue du Picouveau, 13260 Cassis, France","Parking gratuit,Connexion Wi-Fi gratuite,Chamb...",L’hébergement La douceur de Cassis se trouve à...,https://www.booking.com/hotel/fr/la-douceur-de...
8,59,Marseille,43.301957,5.372294,Marseille City Chambres&Appartements,9.4,865.0,"26 Boulevard des Dames, 13002 Marseille, France","Connexion Wi-Fi rapide gratuite (98 Mb/s),Cham...",Le Marseille City Chambres&Appartements propos...,https://www.booking.com/hotel/fr/marseillecity...
9,19,Saintes Maries de la mer,43.467296,4.413867,Maison l'Etrier,9.3,346.0,"chemin bas des launes 373, 13460 Les Saintes-M...","Piscine extérieure,Parking gratuit,Connexion W...","L’établissement Maison l'Etrier se situe à 2,7...",https://www.booking.com/hotel/fr/l-39-etrier.f...


In [9]:
# Plot the map of the top 20 hotels in the top 5 places
fig = px.scatter_map(top20_hotels, lat="hotel_lat",
                        lon="hotel_lon",
                        color='hotel_review',
                        size='hotel_nbr_review',
                        size_max=35,  
                        title='Top 20 hotels in the top 5 places where the weather is best in France in the next 7 days',
                        color_continuous_scale="Viridis",
                        opacity=0.7,
                        hover_name='hotel_name',
                        width=1200, height=900,  
                        zoom=5)  
fig.update_layout(margin={"r":10,"t":100,"l":10,"b":10})
fig.show()
fig.write_html("top_20_hotels.html")